# 부록 10.1: 프롬프트 체이닝

- [수업](#lesson)
- [예제 플레이그라운드](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정합니다.

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(messages, system_prompt=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## 수업

"쓰기는 다시 쓰기다"라는 말이 있습니다. **Claude에게 다시 해볼 것을 요청하면 종종 응답의 정확성을 높일 수 있습니다**!

Claude에게 "다시 생각해 보라"고 요청하는 방법은 여러 가지가 있습니다. 사람에게 작업을 다시 확인하라고 요청하는 자연스러운 방식은 일반적으로 Claude에게도 통합니다. (프롬프트 체이닝을 언제 그리고 어떻게 사용해야 하는지에 대한 자세한 예시는 [프롬프트 체이닝 문서](https://docs.anthropic.com/claude/docs/chain-prompts)를 확인하세요.)

### 예시

이 예시에서 Claude에게 열 단어를 만들어 보라고 요청합니다... 하지만 그중 하나 이상은 실제 단어가 아닙니다.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {"role": "user",
     "content": [{"text": first_user}]}
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

**Claude에게 답변을 더 정확하게 하라고 요청하면** 오류를 수정합니다! 

아래에서는 위의 Claude의 잘못된 응답을 가져와 대화에 다른 차례를 추가하여 이전 답변을 수정하라고 요청했습니다.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

하지만 Claude가 우리가 말한 대로 답변을 수정하는 것일까요? 이미 정확한 답변으로 시작한다면 어떨까요? Claude는 자신감을 잃을까요? 여기서는 `first_response` 자리에 정확한 응답을 넣고 다시 한번 확인해 달라고 요청했습니다.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

위 블록에서 몇 번 응답을 생성해 보면 Claude가 대부분의 경우에는 단어를 그대로 두지만, 모든 단어가 이미 정확함에도 불구하고 가끔 단어를 변경하는 것을 알 수 있습니다. 이를 어떻게 완화할 수 있을까요? 8장에서 설명한 대로 Claude에게 선택권을 주면 됩니다! 한 번 더 시도해 봅시다.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

위 코드에서 몇 번 응답을 생성해 보면 Claude가 이제 훨씬 더 잘 고수하는 것을 볼 수 있습니다.

프롬프트 체이닝을 사용하여 **Claude에게 응답을 개선하라고 요청**할 수도 있습니다. 아래에서는 Claude에게 먼저 이야기를 쓰고 나서 작성한 이야기를 개선하라고 요청했습니다. 개인적인 취향에 따라 다를 수 있지만, 많은 사람들이 Claude의 두 번째 버전이 더 낫다고 동의할 것입니다.

먼저 Claude의 첫 번째 이야기 버전을 생성해 봅시다.

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
         "content": [{"text": first_user}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

이제 Claude에게 첫 번째 초안을 개선하라고 요청해 봅시다.

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

이러한 대체 방식은 매우 강력합니다. 우리는 대체 자리 표시자를 사용하여 목록, 단어, Claude의 이전 응답 등을 전달했습니다. 또한 대체를 사용하여 "함수 호출"이라고 하는 것을 수행할 수 있습니다. 즉, Claude에게 어떤 기능을 수행하라고 요청한 다음 그 결과를 가져와 Claude에게 결과로 더 많은 작업을 수행하라고 요청하는 것입니다. 다른 대체와 마찬가지로 작동합니다. 다음 부록에서 자세히 다룹니다.

아래는 Claude에 대한 한 번의 호출 결과를 가져와 다른 더 긴 호출에 플러그인하는 또 다른 예시입니다. 먼저 첫 번째 프롬프트(이번에는 Claude의 응답을 미리 채워 넣었습니다)로 시작해 봅시다.

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

이 이름 목록을 다른 프롬프트에 전달해 봅시다.

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill + "\n" + first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

이제 프롬프트 체이닝에 대해 배웠으니, 부록 10.2로 가서 프롬프트 체이닝을 사용하여 함수 호출을 구현하는 방법을 배우세요.

---

## 예제 실습 공간

이 공간에서는 이 수업에서 보여준 프롬프트 예제를 자유롭게 실험하고 프롬프트를 조정하여 Claude의 응답에 어떤 영향을 미치는지 확인할 수 있습니다.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {"role": "user",
     "content": [{"text": first_user}]}
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
         "content": [{"text": first_user}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill + "\n" + first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))